In [1]:
import pandas as pd
from openpyxl import load_workbook
from statsmodels.stats.multitest import multipletests

input_workbook = 'combined_scoary_results.xlsx'
output_workbook = 'processed_combined_scoary_results.xlsx'


def process_group(df):
    pvals = df['Empirical_p']
    adj_pvals = multipletests(pvals, method='fdr_bh')[1]
    df['adj_p.value'] = adj_pvals
    return df

with pd.ExcelWriter(output_workbook, engine='openpyxl') as writer:
    workbook = load_workbook(input_workbook)
    for sheet_name in workbook.sheetnames:
        if sheet_name == 'commands':
            continue

        df = pd.read_excel(input_workbook, sheet_name=sheet_name)
        df = df[['comparison-type', 'niche', 'genes', 'Odds_ratio', 'Empirical_p']]

        groups = df.groupby(['comparison-type', 'niche'])

        processed_groups = []
        for _, group_df in groups:
            processed_groups.append(process_group(group_df))

        df = pd.concat(processed_groups)

        def map_niche(niche):
            if niche == 'rhizo':
                return 'rhizo & soil'
            elif niche == 'phyllo':
                return 'phyllo & soil'
            return niche

        df['niche'] = df['niche'].apply(map_niche)
        
        df['stats_method'] = 'scoary'

        df['raw_enriched'] = df.apply(lambda x: 1 if x['comparison-type'] == 'raw' and x['adj_p.value'] < 0.1 and x['Odds_ratio'] > 1 else 0, axis=1)
        df['raw_depleted'] = df.apply(lambda x: 1 if x['comparison-type'] == 'raw' and x['adj_p.value'] < 0.1 and x['Odds_ratio'] < 1 else 0, axis=1)
        df['binary_enriched'] = df.apply(lambda x: 1 if x['comparison-type'] == 'binary' and x['adj_p.value'] < 0.1 and x['Odds_ratio'] > 1 else 0, axis=1)
        df['binary_depleted'] = df.apply(lambda x: 1 if x['comparison-type'] == 'binary' and x['adj_p.value'] < 0.1 and x['Odds_ratio'] < 1 else 0, axis=1)

        df = df.groupby(['genes', 'niche', 'stats_method']).agg({
            'raw_enriched': 'sum',
            'raw_depleted': 'sum',
            'binary_enriched': 'sum',
            'binary_depleted': 'sum'
        }).reset_index()

        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f'Processed all sheets and saved to {output_workbook}')


Processed all sheets and saved to processed_combined_scoary_results.xlsx


In [2]:
import pandas as pd

input_workbook = 'processed_combined_scoary_results.xlsx'
output_workbook = 'renamed_processed_combined_scoary_results.xlsx'

# Load the workbook and get sheet names
xl = pd.read_excel(input_workbook, sheet_name=None)

with pd.ExcelWriter(output_workbook, engine='openpyxl') as writer:
    for sheet_name, df in xl.items():
        # Rename the 'genes' column to 'gene'
        df.rename(columns={'genes': 'gene'}, inplace=True)

        # Save the updated dataframe to the output workbook
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print(f'Renamed "genes" column to "gene" in all sheets and saved to {output_workbook}')


Renamed "genes" column to "gene" in all sheets and saved to renamed_processed_combined_scoary_results.xlsx


In [3]:
import pandas as pd

input_workbook = 'all_tests_combined.xlsx'
output_workbook = 'updated_all_tests_combined.xlsx'

# Load the workbook and get sheet names
xl = pd.read_excel(input_workbook, sheet_name=None)

# Create an empty DataFrame to store the concatenated data
combined_df = pd.DataFrame()

with pd.ExcelWriter(output_workbook, engine='openpyxl') as writer:
    for sheet_name, df in xl.items():
        # Add a new column 'Order' with the value set to the sheet name
        df['Order'] = sheet_name
        
        # Save the updated dataframe to the output workbook
        df.to_excel(writer, sheet_name=sheet_name, index=False)

        # Add the updated dataframe to the combined dataframe
        combined_df = combined_df.append(df, ignore_index=True)

    # Save the combined dataframe to the output workbook
    combined_df.to_excel(writer, sheet_name='combined', index=False)

print(f'Added "Order" column and combined all sheets into a single sheet named "combined" in {output_workbook}')



/var/folders/fz/pjh7t0w51t5d366ytggct8k40000gn/T/ipykernel_7435/3242394451.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/var/folders/fz/pjh7t0w51t5d366ytggct8k40000gn/T/ipykernel_7435/3242394451.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/var/folders/fz/pjh7t0w51t5d366ytggct8k40000gn/T/ipykernel_7435/3242394451.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  combined_df = combined_df.append(df, ignore_index=True)
/var/folders/fz/pjh7t0w51t5d366ytggct8k40000gn/T/ipykernel_7435/3242394451.py:21: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future ve

Added "Order" column and combined all sheets into a single sheet named "combined" in updated_all_tests_combined.xlsx


In [5]:
import pandas as pd

input_workbook = 'updated_all_tests_combined.xlsx'
output_workbook = 'aggregated_all_tests_combined.xlsx'

# Load the 'combined' sheet
df = pd.read_excel(input_workbook, sheet_name='combined')

# Replace the niche column values
df['niche'] = df['niche'].replace({'phyllo &soil': 'phyllo & soil', 'rhizo & zoil': 'rhizo & soil'})

# Group the rows by niche, Order, gene and aggregate the columns
aggregated_df = df.groupby(['niche', 'Order', 'gene']).agg(
    {'raw_enriched': 'sum',
     'raw_depleted': 'sum',
     'binary_enriched': 'sum',
     'binary_depleted': 'sum'}
).reset_index()

# Save the aggregated dataframe as a new sheet in the output workbook
with pd.ExcelWriter(output_workbook, engine='openpyxl') as writer:
    aggregated_df.to_excel(writer, sheet_name='aggregated', index=False)

print(f'Aggregated the "combined" sheet and saved as "aggregated" sheet in {output_workbook}')


Aggregated the "combined" sheet and saved as "aggregated" sheet in aggregated_all_tests_combined.xlsx
